In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

In [ ]:
#https://www.delish.com/cooking/recipe-ideas/a24892347/how-to-make-a-smoothie/
#https://www.pressurecookrecipes.com/instant-pot-spaghetti-bolognese/

hdr = {'User-Agent' : 'Mozilla/5.0'}
loop = True
while loop:
  link = input('Enter recipe link')
  print('Copy and paste into spreadsheet under the \'Ingredients\' column for the proper row. Delete/Edit any necessary entries')

  url = link
  req = Request(url, headers=hdr)
  page = urlopen(req)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")

  x = soup.find(class_=re.compile("ingredient")).get_text('|')
  x=x.split('|')
  ingArr = [item for item in x if item.replace(" ","").isalpha()]
  print(', '.join(ingArr))

  end = input('Are you done inputting recipes? y/n')
  if end=='y':
    loop=False

Enter recipe linkhttps://www.pressurecookrecipes.com/instant-pot-spaghetti-bolognese/
Copy and paste into spreadsheet under the 'Ingredients' column for the proper row. Delete/Edit any necessary entries
Ingredients, spaghetti, pound, ground beef, small onion, cloves, garlic, rib, celery, white mushrooms, Sherry wine, A pinch , dried oregano, A pinch, dried basil, Kosher salt and black pepper to taste, tablespoons, olive oil, Chicken Stock Mixture, cup, unsalted chicken stock, cups, water, tomato paste, tablespoons, regular soy sauce, tablespoon, fish sauce, tablespoon, Worcestershire sauce, Equipment, Instant Pot Pressure Cooker
Are you done inputting recipes? y/nn
Enter recipe linkhttps://www.pressurecookrecipes.com/instant-pot-spaghetti-bolognese/
Copy and paste into spreadsheet under the 'Ingredients' column for the proper row. Delete/Edit any necessary entries
Ingredients, spaghetti, pound, ground beef, small onion, cloves, garlic, rib, celery, white mushrooms, Sherry wine, A pinch

latent dirichlet allocation

In [ ]:
%pip install lda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.8/349.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as lda2
import lda
import pandas as pd
import numpy as np
import io

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
data = pd.read_csv(io.BytesIO(uploaded['Smoothie Test Dataset - Sheet1(7).csv']))
df = pd.DataFrame(data)
print(df)

NameError: ignored

In [ ]:
#userIn = input()
flag = True
userRec = []
userIngr = []

while flag:
  print("Enter Recipe Name: ")
  userIn = input()
  userRec.append(userIn)
  #print(userRec)
  print("Enter the Ingredients of the recipe separated by commas")
  userIn = input()
  userIngr.append(userIn.split(", "))
  flag = False

userRec
userIngr

Enter Recipe Name: 
egg
Enter the Ingredients of the recipe separated by commas
   egg  , Bacon  


AttributeError: ignored

In [ ]:
gsheetid = "1jMl5uAKJs1AWDU4D-gt5yQzklx7dPlRtbFDoreAdW1c"
sheet_name = "Smoothie Test Dataset"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
url = url.replace(" ", "%20")
df = pd.read_csv(url)
df

,Smoothie,Ingredients
0,Strawberry Smoothie,"Strawberry, Milk, Banana"
1,Mango Smoothie,"Mango, Milk, Banana, Lactaid"
2,Watermelon Smoothie,"Watermelon, Lactaid, Greek Yogurt"
3,Strawberry Smoothie,"Strawberry, Milk, Banana"
4,Watermelon,"Watermelon, Cola"
5,Cola,"Cola, Sugar"


In [ ]:
df['Ingredients'] = df['Ingredients'].apply(lambda x: ''.join(x)) #prepping ingredients column for feature extraction
l = df['Ingredients'].values.tolist()
l

['Strawberry, Milk, Banana',
 'Mango, Milk, Banana, Lactaid',
 'Watermelon, Lactaid, Greek Yogurt',
 'Strawberry, Milk, Banana',
 'Watermelon, Cola',
 'Cola, Sugar']

In [ ]:
vec = CountVectorizer(analyzer='word', ngram_range=(1,1)) #Count Vectorization to extract features from the ingredients
X = vec.fit_transform(l)
vec.get_feature_names_out() #showing the features that were grabbed

array(['banana', 'cola', 'greek', 'lactaid', 'mango', 'milk',
       'strawberry', 'sugar', 'watermelon', 'yogurt'], dtype=object)

In [ ]:
topics = 2 #minimum number of clusters being 2. Assumes that the recipes given are different enough for at least 2 groups.
highestll = -1000000000 #The higher the log liklihood, the better chance that the number of groups is correct. We start with an extremely small log liklihood so that it can be updated on the first pass in the loop.
for x in range(topics, 20): #Loop through 2-20, testing for different number of topics/groups that yield a higher log liklihod. When the log liklihood is at its peak, we know that the number of groups is the best for that dataset.
  model = lda.LDA(n_topics=x, random_state=1)
  model.fit(X)
  ll = model.loglikelihood()
  if(ll>highestll):
      highestll = ll
  else:
      topics = x-1 #once the log liklihood is lower than the one previously, we know that the LL has hit its peak. Thus we use the model with the x-1 (being the peak) topics.
      model = lda.LDA(n_topics=x-1, random_state=1)
      model.fit(X)
      break

In [ ]:
df.iloc[1,0]

'Mango Smoothie'

In [ ]:
doc_topic = model.doc_topic_

for i in range(len(l)):
    print(f'{df.iloc[i,0]}:', doc_topic[i].argmax())


Strawberry Smoothie: 1
Mango Smoothie: 1
Watermelon Smoothie: 0
Strawberry Smoothie: 1
Watermelon: 0
Cola: 0
